In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Import Income Data
Income data from https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6

In [2]:
income_raw = pd.read_csv("IncomeByCounty.csv")

In [3]:
income_raw.info()
income_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9414 entries, 0 to 9413
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GeoFips      9414 non-null   int64 
 1   GeoName      9414 non-null   object
 2   LineCode     9414 non-null   int64 
 3   Description  9414 non-null   object
 4   2019         9414 non-null   object
dtypes: int64(2), object(3)
memory usage: 367.9+ KB


,GeoFips,GeoName,LineCode,Description,2019
0,1001,"Autauga, AL",1,Personal income (thousands of dollars),2453617
1,1001,"Autauga, AL",2,Population (persons) 1/,55869
2,1001,"Autauga, AL",3,Per capita personal income (dollars) 2/,43917
3,1003,"Baldwin, AL",1,Personal income (thousands of dollars),10600260
4,1003,"Baldwin, AL",2,Population (persons) 1/,223234


## Income Data Cleaning
Row data different depending on linecode, split different linecodes into income and population rows
Need county population and county income as columns and can remove other columns

In [4]:
#Seperate data by linecode value
group = income_raw.groupby("LineCode")

counties = group.get_group(1)["GeoName"].reset_index()
del counties["index"]


income_data = group.get_group(1)["2019"].reset_index()
del income_data["index"]

population_data = group.get_group(2)["2019"].reset_index()
del population_data["index"]

In [5]:
county_data = pd.DataFrame(counties)
county_data = county_data.rename(columns={"GeoName": "County"})

county_data["Total Income"] = income_data
county_data["Population"] = population_data

county_data

,County,Total Income,Population
0,"Autauga, AL",2453617,55869
1,"Baldwin, AL",10600260,223234
2,"Barbour, AL",882834,24686
3,"Bibb, AL",710451,22394
4,"Blount, AL",2105550,57826
...,...,...,...
3133,"Sweetwater, WY",2423098,42343
3134,"Teton, WY",5392611,23464
3135,"Uinta, WY",847702,20226
3136,"Washakie, WY",395988,7805


## Remove "(NA)" from dataset
To allow datatype change (There is probably a better way to do this)

In [6]:
county_data = county_data.replace("(NA)", "1")

## Convert income and population columns to numeric
For calculation of income per capita


In [7]:
county_data["Total Income"] = pd.to_numeric(county_data["Total Income"])
county_data["Population"] = pd.to_numeric(county_data["Population"])

county_data.dtypes

County          object
Total Income     int64
Population       int64
dtype: object

## Create income per capita column
Add calculated income per capita in thousands as a column

In [11]:
percapitaincome = county_data.apply(lambda row: row["Total Income"] / row["Population"], axis = 1).reset_index()
del percapitaincome["index"]
county_data["Income per Capita"] = percapitaincome
county_data.to_csv("CleanedIncomePerCapita.csv")
county_data

,County,Total Income,Population,Income per Capita
0,"Autauga, AL",2453617,55869,43.917324
1,"Baldwin, AL",10600260,223234,47.484971
2,"Barbour, AL",882834,24686,35.762537
3,"Bibb, AL",710451,22394,31.725060
4,"Blount, AL",2105550,57826,36.411822
...,...,...,...,...
3133,"Sweetwater, WY",2423098,42343,57.225468
3134,"Teton, WY",5392611,23464,229.824881
3135,"Uinta, WY",847702,20226,41.911500
3136,"Washakie, WY",395988,7805,50.735170
